### Interactive Maps: An exploration with Folium

The world of online maps is a playground for the curious! Digital cartographies offer more than just static images such as interactive zooming, panning across areas and retrieving results from various queries.This is typically done through the use of of JavaScipt.In our case, we find [Leaflet](https://leafletjs.com/), a lightweight and user-friendly library, and [OpenLayers](https://openlayers.org/), known for its extensive mapping features. However we  won't delve into the JavaScript coding. This is a Python course and we will be using  the [Folium](https://python-visualization.github.io/folium/) Python package, a tool that bridges the gap between the Leaflet library and Python's capabilities. With Folium, we'll transform data housed in [geopandas.GeoDataFrames](https://geopandas.org/en/stable/docs/user_guide/data_structures.html) into interactive Python elements.


**Find more information about the capabilities of the *Folium* package on its official web pages:**

- **Documentation**: [Explore the official documentation](https://python-visualization.github.io/folium/) for comprehensive details on Folium's features and how to use them.
- **Example gallery**: [View the example gallery](https://nbviewer.org/github/python-visualization/folium/tree/main/examples/) to see Folium in action, with various maps and visualizations created using the package.
- **Quickstart tutorial**: [Start with the Quickstart tutorial](https://python-visualization.github.io/folium/quickstart.html#Getting-Started) for a hands-on introduction to creating your first interactive map with Folium.

Dive into these resources to unlock the full potential of interactive map-making with Folium in your projects.


### Create a simple interactive web map



Our journey into the world of interactive web mapping begins with a simple yet foundational step: creating a basic web map that features solely a base map. This initial exercise is designed to familiarize us with Folium's syntax, guiding us through the essential steps and conventions used in the library.

**Creating a Map with Folium:**

1. **Initialize a Map Object**: We start by crafting a `folium.Map` object. This involves specifying a central point (`location`) around which the map will be centered and setting an initial zoom level (ranging from `0` to `20`). This zoom level dictates how close or far away the map appears upon loading.

2. **Add a Scale Bar**: By enabling the `control_scale` parameter (`control_scale=True`), we instruct Folium to display a scale bar on the map. This scale bar adds a valuable reference for understanding distances and the scale of map features.

This process lays the groundwork for our exploration of interactive web mapping, setting the stage for more complex and data-rich visualizations.


In [ ]:
import pathlib
NOTEBOOK_PATH = pathlib.Path().resolve()
DATA_DIRECTORY = NOTEBOOK_PATH / "data"

# We will export HTML pages during this lesson,
# let’s also prepare an output directory for them:
HTML_DIRECTORY = NOTEBOOK_PATH / "html"
HTML_DIRECTORY.mkdir(exist_ok=True)

In [ ]:
import folium

interactive_map = folium.Map(
    location=(59.33, 18.06),
    zoom_start=10,
    control_scale=True
)

interactive_map

In [ ]:
interactive_map.save(HTML_DIRECTORY / "base-map.html")

### Change the base map



Folium's flexibility extends to its ability to use various base layers, not just the default OpenStreetMap. When creating a `folium.Map`, the `tiles` parameter opens the door to customization, allowing you to either select from built-in map providers or specify a custom tileset URL to tailor the map's appearance to your needs.

**Adjusting Map Center and Zoom Level:ding. By adjusting the `location` parameter, you can set a new center for your map exploration. Similarly, changing the `zoom_start` parameter lets you fine-tune the initial zoom level, providing a closer look or a broader overview based on your prefereploring.


In [ ]:
interactive_map = folium.Map(
    location=(59.33, 18.06),
    zoom_start=12,
    tiles='CartoDB positron'
)
interactive_map

### Add a point marker



To enrich your Folium map with interactive markers, you can easily integrate a `folium.Marker` into your map. These markers not only pinpoint specific locations but also offer customizable visual styles and interactive toolti.t.

**How to Add a Marker:**

1. **Create a Marker Object**: Begin by creating a `folium.Marker` instance. This object requires a location parameter, defined by latitude and longitude coordinates, to place the marker on the map.

2. **Customize the Marker Icon**: For a personalized touch, customize your marker's appearance by supplying a `folium.Icon` object to the `icon` parameter. This customization can include changing the marker's color, icon style, and more, allowing you to match the marker's look to your map's theme or purpose.

3. **Add Interactive Tooltips**: To make your markers informative, you can add a tooltip that displays text when users hover their mouse over the marker. Set the `tooltip` parameter with your desired text to provide additional context or information about the marked location.

With these simple steps, you can enhance the interactivity and informational value of your Folium map.

In [ ]:
interactive_map = folium.Map(
    location=(59.33, 18.06),
    zoom_start=12
)

KTH = folium.Marker(
    location=(59.3499, 18.0703),
    tooltip="KTH",
    icon=folium.Icon(color="green", icon="ok-sign")
)
KTH.add_to(interactive_map)

interactive_map

**Incorporating a Layer of Points with Folium**

Folium extends its capabilities beyond single markers, allowing for the addition of spatial layers, such as those from `geopandas.GeoDataFrames`. It wraps Leaflet's `geoJSON` layers within its `folium.features.GeoJson` class. This functionality lets us seamlessly integrate geo-data frames as layers into our maps.


In [1]:
import geopandas

addresses = geopandas.read_file(DATA_DIRECTORY / "stockholm_addresses.gpkg")
addresses

NameError: name 'DATA_DIRECTORY' is not defined

In [ ]:
interactive_map = folium.Map(
    location=(59.3499, 18.0703),
    zoom_start=12
)

addresses_layer = folium.features.GeoJson(
    addresses,
    name="Landmarks in Stockholm",
    tooltip=folium.features.GeoJsonTooltip(fields=['address'], labels=False)  

)
addresses_layer.add_to(interactive_map)

interactive_map

### Add a polygon layer

In the following section we are going to revisit another data set with which we have worked before: DeSO administrative delineation over Stockholm from the SCB open data WFS endpoint:

In [ ]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

# Define the base URL for the WFS request
WFS_BASE_URL = (
    "http://geodata.scb.se/geoserver/stat/wfs"
    "?service=wfs"
    "&version=1.1.0"
    "&request=GetFeature"
    "&srsName=EPSG:3006"
    "&typeName={layer:s}"
)

# Specify the layer you're interested in
# For this example, we use "DeSO.2018" as mentioned in the document
layer_name = "DeSO.2018"

# Fetch the data using GeoPandas
deso = geopandas.read_file(
    WFS_BASE_URL.format(layer=layer_name)
).set_crs("EPSG:3006")

# Display the first few rows of the data
print(deso.head())

Let’s first clean the data frame: drop all columns we don’t need, and rename the remaining ones to English.

In [ ]:
deso = deso[["deso", "kommunnamn","kommun", "geometry"]]
deso = deso.rename(columns={
    "kommunnamn": "municipality"
})

We will use a similar approach with a `GeoJson`layer to display the DeSO/Munipality polygons. 

In [ ]:

deso["deso"] = deso.index.astype(str)


In [ ]:

# Function to style the features based on the 'municipality' data
def style_function(feature):
    return {
        'fillColor': '#ffaf00', 
        'color': 'black',
        'weight': 2,
        'dashArray': '5, 5'
    }

# Create the map
interactive_map = folium.Map(
    location=(59.3499, 18.0703),
    zoom_start=12
)

# Add GeoJson layer
population_grid_layer = folium.GeoJson(
    data=deso,
    style_function=style_function,  # Apply styling to each feature
    tooltip=folium.features.GeoJsonTooltip(fields=['deso', 'municipality']),  
).add_to(interactive_map)

interactive_map


In [ ]:
ssl._create_default_https_context = ssl._create_unverified_context

# Define the base URL for the WFS request
WFS_BASE_URL = (
    "http://geodata.scb.se/geoserver/stat/wfs"
    "?service=wfs"
    "&version=1.1.0"
    "&request=GetFeature"
    "&srsName=EPSG:3006"
    "&typeName={layer:s}"
)

#Similar process for the urban layer
tatorter_1980 = geopandas.read_file(
    WFS_BASE_URL.format(layer= "stat:Tatorter.1980")
).set_crs("EPSG:3006")

# Display the first few rows of the data
print(tatorter_1980.head())

In [ ]:
urban_areas = tatorter_1980[["kommunnamn", "bef", "geometry"]]
urban_areas = urban_areas.rename(columns={
    "bef": "population"
})

In [ ]:
urban_areas.head()

**Creating Thematic Maps with `folium.Choropleth`**

To visualize the population distribution, we'll employ the `folium.Choropleth` class. Choropleth maps extend beyond mere representations of polygon geometries — akin to those we showcased using the `folium.features.GeoJson` layer for addressing points previously. This powerful class adeptly manages data categorization, incorporates a legend, and executes several minor yet impactful tasks to  generate robust thematic maps.

**Key Expectations of the `folium.Choropleth` Class:**

- **Distinct Data Handling**: It requires a dataset featuring an explicitly defined, string-type index column. This prerequisite stems from its design to consider geospatial and thematic inputs as two distinct datasets that necessitate merging.
  
  To illustrate, we will delve into specifying both `geo_data` and `data` in subsequent steps.

- **Efficient Data Indexing Strategy**: An effective method to meet this requirement involves duplicating the DataFrame's index into a new column, named `id` for instance. This technique ensures a seamless integration of geospatial shapes with their corresponding thematic data, enabling a richer and more informative map visualization.

By following these guidelines, we can leverage `folium.Choropleth` to its full potential, creating maps that are not only visually appealing but also rich in information and easy to interpret.


In [ ]:
urban_areas["id"] = urban_areas.index.astype(str)

In [ ]:
urban_areas.head()

**Adding a Polygon Choropleth Layer with Folium**

Creating a choropleth layer involves integrating both geospatial and thematic datasets into a map. Folium's architecture, while robust, requires careful attention to the parameters we provide:

- **`geo_data` and `data`**: These parameters represent the geospatial and thematic datasets, respectively. Interestingly, both can point to the same `geopandas.GeoDataFrame` if it contains all necessary information.

- **`columns`**: This is a tuple specifying two critical columns within the `data` dataset:
  1. A unique index column that uniquely identifies each feature.
  2. The column containing the thematic data intended for visualization.

- **`key_on`**: This parameter designates which column in `geo_data` will be used to join with the `data`. It effectively matches the first element of the `columns` tuple, ensuring a proper linkage between the geospatial shapes and the thematic data.


In [2]:
interactive_map = folium.Map(
    location=(59.3499, 18.0703),
    zoom_start=12
)

urban_areas_layer = folium.Choropleth(
    geo_data=urban_areas,
    data=urban_areas,
    columns=("id", "population"),
    key_on="feature.id"
)
urban_areas_layer.add_to(interactive_map)

interactive_map

NameError: name 'folium' is not defined

In [ ]:
interactive_map = folium.Map(
    location=(59.3499, 18.0703),
    zoom_start=12
)

urban_areas_layer = folium.Choropleth(
    geo_data=urban_areas,
    data=urban_areas,
    columns=("id", "population"),
    key_on="feature.id",
    bins=8,
    fill_color="BuPu",
    line_weight=1,
    legend_name="Urban Population, 1980",
    highlight=True
)
urban_areas_layer.add_to(interactive_map)

interactive_map

In [ ]:

from folium.features import GeoJsonTooltip

# Your existing Choropleth map creation code
interactive_map = folium.Map(
    location=(59.3499, 18.0703),
    zoom_start=12
)

urban_areas_layer = folium.Choropleth(
    geo_data=urban_areas,
    data=urban_areas,
    columns=("id", "population"),
    key_on="feature.id",
    bins=8,
    fill_color="BuPu",
    line_weight=0.5,
    legend_name="Urban Population, 1980",
    highlight=True
)
urban_areas_layer.add_to(interactive_map)

# Adding mouseover tooltip

def style_function(feature):
    return {
        'fillColor': '#ffffff',  # Transparent fill
        'color': 'black',  # Border color
        'weight': 0.5,  # Border width
        'fillOpacity': 0.0,  # Make fill fully transparent
    }
tooltip_layer = folium.GeoJson(
    urban_areas,
    style_function=style_function,  # Apply custom style
    name="Urban Areas",
    tooltip=GeoJsonTooltip(
        fields=['id', 'population'],
        aliases=['ID: ', 'Population: '], 
        localize=True,
         sticky=False,
        labels=True,
        max_width=50,
    )
)

tooltip_layer.add_to(interactive_map)

# Display the map
interactive_map


Folium is just one of many packages that provide an easy way to create interactive maps using data stored in (geo-)pandas data frames. Other interesting libraries include:

- [GeoViews](https://geoviews.org/), offering high-level building blocks for building complex geospatial plots more simply.
- [Mapbox GL for Jupyter](https://github.com/mapbox/mapboxgl-jupyter), integrating the powerful Mapbox GL JS library with the Jupyter notebook environment for interactive mapping.
- [Bokeh](https://bokeh.org/), a flexible Python library for creating interactive plots and maps that can easily be embedded in web applications.
- [Plotly Express](https://plotly.com/python/plotly-express/), part of the Plotly graphing library, providing a simple syntax for creating complex plots including maps with minimal code.

Each of these libraries has its unique features and strengths, making them worth exploring for different interactive mapping needs and scenarios.


In [ ]:
interactive_map.save(HTML_DIRECTORY / "fancy-map.html")

This lesson has adapted or reused material from the https://autogis-site.readthedocs.io/en/2022/course-info/license.html course under a Attribution-ShareAlike 4.0 International license (https://creativecommons.org/licenses/by-sa/4.0/)